In [1]:
import tensorflow as tf
import gpflow
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [12]:
class SpectralMixture(gpflow.kernels.Stationary):
    def __init__(self, q=1, float32=False):
        gpflow.kernels.Stationary.__init__(self, input_dim=1, active_dims=[0], lengthscales=1.0)
        self.q = q
        random_params = np.random.random((3, q))
        if float32:
            random_params = random_params.astype(np.float32)
        self.w = gpflow.ParamList([gpflow.Param(random_params[0][i],
                                                transform=gpflow.transforms.positive) for i in range(q)])
        self.sigma_upp = gpflow.ParamList([gpflow.Param(random_params[1][i],
                                                        transform=gpflow.transforms.positive) for i in range(q)])
        self.mu = gpflow.ParamList([gpflow.Param(random_params[2][i]) for i in range(q)])

    def K(self, X, X2=None, presliced=False):
        if X2 is None:
            X2 = X
        tau = X-tf.transpose(X2)
        ans = 0
        for i in range(self.q):
            #print(tau.dtype, self.sigma_upp[i].dtype, self.mu[i].dtype, self.w[i].dtype)
            if self.sigma_upp[i].dtype != tau.dtype:
                a = tf.exp(tf.negative(tf.cast(self.sigma_upp[i], tf.float32)*tf.square(tau)))
                b = tf.cos(tf.cast(self.mu[i], tf.float32) * tau)
                ans += tf.cast(self.w[i], tf.float32) * a * b
            else:
                ans += self.w[i]* tf.exp(tf.negative(self.sigma_upp[i]*tf.square(tau))) * tf.cos(self.mu[i] * tau)
        return ans

    def Kdiag(self, X):
        ans = 0
        for i in range(self.q):
            ans += self.w[i]
        return np.sum(self.w)

In [3]:
class MOSpectralMixture(gpflow.kernels.Stationary): # stationary, eucl_dist
    def __init__(self, m):
        gpflow.kernels.Stationary.__init__(self, input_dim=1, active_dims=[0], lengthscales=1.0)
        self.m = m
        self.w = gpflow.Param(np.random.random((m,)), transform=gpflow.transforms.positive)
        self.sigma = gpflow.Param(np.random.random((m,)), transform=gpflow.transforms.positive)
        self.mu = gpflow.Param(np.random.random((m,)))
        self.theta = gpflow.Param(np.random.random((m,)))
        self.phi = gpflow.Param(np.random.random((m,)))
    
    def get_correlated_parameters(self, I, len_I, I2, len_I2):
        '''Returns matrices with the corresponding correleted parameters in each position'''
        i = tf.tile(I, [1, len_I2])
        j = tf.tile(tf.transpose(I2), [len_I, 1])
        sigma_i = tf.gather(self.sigma, i)
        sigma_j = tf.gather(self.sigma, j)
        sigma_ij = tf.div(2*sigma_i*sigma_j, sigma_i + sigma_j)
        mu_i = tf.gather(self.mu, i)
        mu_j = tf.gather(self.mu, j)
        mu_ij = tf.div(sigma_i*mu_j + sigma_j*mu_i, sigma_i + sigma_j)
        w_i = tf.gather(self.w, i)
        w_j = tf.gather(self.w, j)
        w_ij = w_i*w_j*tf.exp(tf.div(-0.25*tf.square(mu_i - mu_j), sigma_i + sigma_j))
        theta_ij = tf.gather(self.theta, i) - tf.gather(self.theta, j)
        phi_ij = tf.gather(self.phi, i) - tf.gather(self.phi, j)
        return sigma_ij, mu_ij, w_ij, theta_ij, phi_ij

    def K(self, X, X2=None, presliced=False):
        if X2 is None:
            X2 = X
        points_x = tf.reshape(X[:,0], (-1,1))
        points_x2 = tf.reshape(X2[:,0], (-1,1))
        tau = tf.tile(points_x, [1, tf.shape(X2)[0]]) -\
              tf.tile(tf.transpose(points_x2), [tf.shape(X)[0], 1])
        I_X = tf.cast(tf.reshape(X[:,1], (-1,1)), dtype=tf.int32)
        I_X2 = tf.cast(tf.reshape(X2[:,1], (-1,1)), dtype=tf.int32)
        sigma_ij, mu_ij, w_ij, theta_ij, phi_ij = self.get_correlated_parameters(I_X, tf.shape(X)[0],
                                                                                 I_X2, tf.shape(X2)[0])
        alpha_ij = tf.scalar_mul((np.float64(2*np.pi))**0.5, w_ij*tf.abs(sigma_ij))
        return alpha_ij*tf.exp(tf.negative((tau + theta_ij)**2*sigma_ij))*\
                tf.cos((tau + theta_ij)*mu_ij + phi_ij)

    def Kdiag(self, X):
        # tau = theta_ij = phi_ij = 0 -> return alpha_ij
        I = tf.transpose(tf.cast(tf.reshape(X[:,1], (-1,1)), dtype=tf.int32))
        w_i = tf.square(tf.gather(self.w, I))
        sigma_i = tf.gather(self.sigma, I)
        return tf.scalar_mul((np.float64(2*np.pi))**0.5, w_i*tf.abs(sigma_i))